# Collaborative Filtering Recommendation Script

### This notebook documents the training of the restaurant recommendation system and model predictions

#### *The first block of code*

In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('raw_ids.csv')
data_names = pd.read_csv('raw.csv')

# Create dictionary of restaurant
rest_dict = data_names[['restaurant_names','restaurant_id']]
rest_dict = rest_dict.drop_duplicates()
rest_dict = rest_dict.set_index('restaurant_id')['restaurant_names'].to_dict()


# Define the reader object
reader = Reader(rating_scale=(1, 5))

# Load the dataset into Surprise's format
dataset = Dataset.load_from_df(data[['users_id', 'restaurant_id', 'ratings']], reader)

# Split the dataset into train and test sets
seed = 7
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=seed)

# Build the collaborative filtering model (SVD algorithm)
model = SVD()

# Train the model on the training set
model.fit(train_set)

# Select a user for whom you want to generate recommendations
user_id = 6
# Get the list of all restaurants in the dataset
all_restaurants = data['restaurant_id'].unique()

# Filter the restaurants that the user has already rated
rated_restaurants = data[data['users_id'] == user_id]['restaurant_id'].values
unrated_restaurants = list(set(all_restaurants) - set(rated_restaurants))

# Create a list to store the predicted ratings for unrated restaurants
predicted_ratings = []

# Generate ratings predictions for unrated restaurants
for restaurant in unrated_restaurants:
    predicted_rating = model.predict(user_id, restaurant).est
    predicted_ratings.append((restaurant, predicted_rating))

# Sort the predicted ratings in descending order
predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Print the top recommended restaurants
top_k = 5  # Number of recommendations to display
print(f'Top {top_k} recommended restaurants for user {user_id}:')
for i, (restaurant, predicted_rating) in enumerate(predicted_ratings[:top_k]):
    print(f'{i+1}. {rest_dict[restaurant]} (predicted rating: {predicted_rating:.2f})')


# Saving and loading the model

In [ ]:
# save the model
import pickle
filename = "recommender_model.sav"
pickle.dump(model, open(filename, 'wb'))

In [1]:
# Load the model from file
with open('recommender_model.sav', 'rb') as file:
    model = pickle.load(file)

# Making restaurant recommendations

In [2]:
import pandas as pd

# Load the dataset
data = pd.read_csv('raw_ids.csv')
data_names = pd.read_csv('raw.csv')

# Create dictionary of restaurant
rest_dict = data_names[['restaurant_names','restaurant_id']]
rest_dict = rest_dict.drop_duplicates()
rest_dict = rest_dict.set_index('restaurant_id')['restaurant_names'].to_dict()

# Select a user for whom you want to generate recommendations
user_id = 599
# Get the list of all restaurants in the dataset
all_restaurants = data['restaurant_id'].unique()

# Filter the restaurants that the user has already rated
rated_restaurants = data[data['users_id'] == user_id]['restaurant_id'].values
unrated_restaurants = list(set(all_restaurants) - set(rated_restaurants))

# Create a list to store the predicted ratings for unrated restaurants
predicted_ratings = []

# Generate ratings predictions for unrated restaurants
for restaurant in unrated_restaurants:
    predicted_rating = model.predict(user_id, restaurant).est
    predicted_ratings.append((restaurant, predicted_rating))

# Sort the predicted ratings in descending order
predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Print the top recommended restaurants
top_k = 5  # Number of recommendations to display
print(f'Top {top_k} recommended restaurants for user {user_id}:')
for i, (restaurant, predicted_rating) in enumerate(predicted_ratings[:top_k]):
    print(f'{i+1}. {rest_dict[restaurant]} (predicted rating: {predicted_rating:.2f})')


Top 5 recommended restaurants for user 599:
1. THE DIRTY KITCHEN (predicted rating: 4.42)
2. Bacchus Restaurant (predicted rating: 4.41)
3. Buffalo Burger - Town Store (predicted rating: 4.38)
4. Mojo Social (predicted rating: 4.38)
5. Fu Gui (predicted rating: 4.37)
